# Gallery Linker Notebook

## References

- [stashdb-performer-gallery](../../gh-stashapp/CommunityScripts/plugins/stashdb-performer-gallery/stashdb-performer-gallery.py)

In [1]:
import stashapi.log as log
from stashapi.stashapp import StashInterface
from stashapi.stashbox import StashBoxInterface
import os
import sys
import requests
import json
import requests
from pathlib import Path
import base64
from urllib.parse import urlparse
from dataclasses import dataclass, field, asdict
from typing import Optional, List, Dict, Any

In [2]:
per_page = 100
request_s = requests.Session()
stash_boxes = {}
scrapers = {}

In [10]:
def build_default_config() -> dict:
    """Build default Stash connection configuration."""
    return {"scheme": "http", "host": "localhost", "port": "9999"}


def update_config_from_params(config: dict, stash_url: str | None, api_key: str | None) -> None:
    """Update configuration with provided URL and API key."""
    if stash_url:
        parsed = urlparse(stash_url)
        config.update(
            {
                "scheme": parsed.scheme or "http",
                "host": parsed.hostname or "localhost",
                "port": str(parsed.port or 9999),
            }
        )
    if api_key:
        config["ApiKey"] = api_key

stash = StashInterface()
config = stash.get_configuration()

dUsing stash (v0.28.1-0) endpoint at http://localhost:9999/graphql


In [4]:
# Linking statistics dataclasses
@dataclass
class LinkingStat:
    count: int = 0
    without_galleries: int | None = field(default=None, metadata={"help": "Number of items without galleries"})
    without_performers: int | None = field(default=None, metadata={"help": "Number of items without performers"})
    without_scenes: int | None = field(default=None, metadata={"help": "Number of items without scenes"})


class SceneStats(LinkingStat):
    def __post_init__(self):
        self.without_scenes = None
        if self.without_performers is None:
            self.without_performers = 0
        if self.without_galleries is None:
            self.without_galleries = 0


class PerformerStats(LinkingStat):
    def __post_init__(self):
        self.without_performers = None
        if self.without_scenes is None:
            self.without_scenes = 0
        if self.without_galleries is None:
            self.without_galleries = 0


class GalleryStats(LinkingStat):
    def __post_init__(self):
        self.without_galleries = None
        if self.without_performers is None:
            self.without_performers = 0
        if self.without_scenes is None:
            self.without_scenes = 0


@dataclass
class LinkingTotals:
    scenes: SceneStats
    performers: PerformerStats
    galleries: GalleryStats



In [5]:
class Filters:
    @staticmethod
    def null_galleries(): return  {"galleries": {"modifier": "IS_NULL"}}
    @staticmethod
    def null_performers(): return {"performers": {"modifier": "IS_NULL"}}
    @staticmethod
    def null_scenes(): return {"scenes": {"modifier": "IS_NULL"}}

FILTERS = Filters()

In [6]:
def get_totals(stash:StashInterface) -> LinkingTotals:
    totals = LinkingTotals(scenes=SceneStats(), performers=PerformerStats(), galleries=GalleryStats())
    totals.galleries.count = stash.find_galleries(get_count=True)[0]
    return totals

totals = get_totals(stash)
# print(totals)

LinkingTotals(scenes=SceneStats(count=0, without_galleries=None, without_performers=None, without_scenes=None), performers=PerformerStats(count=0, without_galleries=None, without_performers=None, without_scenes=None), galleries=GalleryStats(count=3, without_galleries=None, without_performers=None, without_scenes=None))


In [ ]:
def get_scenes_without_galleries(stash: StashInterface):
    """Retrieve the number of scenes without associated galleries."""
    return stash.find_scenes(
        f=FILTERS.null_galleries(), get_count=True
    )[0]

get_scenes_without_galleries(stash)
# print(f"Num of scenes without galleries: {get_num_scenes_without_galleries(stash)}")


4

### CHECKOUT [Ariadne: GraphQL Codegen](https://github.com/mirumee/ariadne-codegen)

Query: 
```graphql
query MissingLinks(
  $sceneWOGalleryfilter: SceneFilterType
  $sceneWOPerformersfilter: SceneFilterType
  $galleryWOPerformerFilter: GalleryFilterType
  $performerWOGalleryFilter: PerformerFilterType
  $performerWOSceneFilter: PerformerFilterType
  $galleryWOSceneFilter:GalleryFilterType) {
    scenesMissingGalleries: findScenes(scene_filter: $sceneWOGalleryfilter) {
      count
    }
    scenesMissingPerformers: findScenes(scene_filter: $sceneWOPerformersfilter) {
      count
    }
    galleriesMissingPerformers: findGalleries(gallery_filter:$galleryWOPerformerFilter){
      count
    }
    galleriesMissingScenes: findGalleries(gallery_filter:$galleryWOSceneFilter){
      count
    }
    performersMissingScenes: findPerformers(performer_filter: $performerWOSceneFilter){
      count
    }
    performersMissingGalleries: findPerformers(performer_filter: $performerWOGalleryFilter){
      count
    }
}
```

Variables:

```gql
{
  "sceneWOGalleryfilter": {
    "galleries": {
      "value": 0, "modifier": "EQUALS"
    }},
  "sceneWOPerformersfilter": {
    "performer_count": {
      "value": 0, "modifier": "EQUALS"
    }},
  "galleryWOSceneFilter":{
    "scenes": {
      "value": 0, "modifier": "EQUALS"
    }},
  "galleryWOPerformerFilter": {
    "performer_count": {
      "value": 0, "modifier": "EQUALS"
    }},
  "performerWOGalleryFilter": {
    "gallery_count": {
      "value": 0, "modifier": "EQUALS"
    }},
  "performerWOSceneFilter": {
    "gallery_count": {
      "value": 0, "modifier": "EQUALS"
    }}
}
```

In [8]:
def get_missing_links(stash):
    """Identify missing links between galleries and scenes/performers."""
    galleries = stash.find_galleries(filter=FILTERS.null_galleries())
    scenes = stash.find_scenes(filter=FILTERS.null_scenes())
    performers = stash.find_performers(filter=FILTERS.null_performers())

    missing_links = {
        "galleries_without_scenes": [],
        "galleries_without_performers": [],
        "scenes_without_galleries": [],
        "scenes_without_performers": [],
        "performers_without_galleries": [],
        "performers_without_scenes": [],
    }

    for gallery in galleries:
        if not gallery.get("scenes"):
            missing_links["galleries_without_scenes"].append(gallery)
        if not gallery.get("performers"):
            missing_links["galleries_without_performers"].append(gallery)
    for scene in scenes:
        if not scene.get("gallery"):
            missing_links["scenes_without_galleries"].append(scene)
        if not scene.get("performers"):
            missing_links["scenes_without_performers"].append(scene)
    for performer in performers:
        if not performer.get("galleries"):
            missing_links["performers_without_galleries"].append(performer)
        if not performer.get("scenes"):
            missing_links["performers_without_scenes"].append(performer)

    return missing_links, galleries, scenes, performers

In [12]:
#ml, gl, sc, pf = get_missing_links(stash)
# print(json.dumps(ml, indent=2, separators=(',', ': ')))
#print("galleries", json.dumps(gl, indent=2, separators=(',', ': ')))
#print("scenes", json.dumps(sc, indent=2, separators=(',', ': ')))
#print("performers", json.dumps(pf, indent=2, separators=(',', ': ')))